In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime
pd.options.display.max_columns = None
from pybaseball import batting_stats, pitching_stats, cache, playerid_lookup, statcast_batter, statcast_pitcher, statcast
import math
cache.enable()
cache.config.cache_type='csv'
cache.config.save()

In [2]:
#keepers Google doc
keepers_url = 'https://docs.google.com/spreadsheets/d/1dwDC2uMsfVRYeDECKLI0Mm_QonxkZvTkZTfBgnZo7-Q/edit#gid=1723951361'

In [3]:
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()
engine = create_engine('sqlite:///fantasy_data.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [4]:
n_teams = 12
tm_players = 23
tm_dollars = 260
player_split = .6
pitcher_split = 1 - player_split
tot_dollars = n_teams * tm_dollars
tot_players = n_teams * tm_players
tot_hitters = n_teams * 14
tot_pitchers = n_teams * 9

drafted_by_pos = {
    'C':n_teams,
    '1B':round(n_teams*1.5),
    '2B':round(n_teams*1.5),
    '3B':math.floor(n_teams*1.5),
    'SS':math.floor(n_teams*1.5),
    'OF':n_teams*5,
    'MI':n_teams,
    'CI':n_teams,
    'DH':n_teams*2, 
    'P':n_teams*9,
    'SP':round(n_teams*6.5),
    'RP':math.floor(n_teams*2.5),
}

In [5]:
def load_id_map():
    player_id_url = 'https://docs.google.com/spreadsheets/d/1JgczhD5VDQ1EiXqVG-blttZcVwbZd5_Ne_mefUGwJnk/pubhtml?gid=0&single=true'
    ids = pd.read_html(player_id_url, header=1)[0]
    ids.drop(columns=['1', 'Unnamed: 9'], inplace=True)
    ids = ids[ids['PLAYERNAME'].notna()]
    return ids

In [6]:
def load_data(kind):
    if kind=='proj':
        atc = pd.read_csv('data/2023-atc-proj-h.csv', encoding="latin-1")
        thebatx = pd.read_csv('data/2023-thebatx-proj-h.csv', encoding="latin-1")
        #dc = pd.read_csv('data/2023-dc-proj-h.csv', encoding="latin-1")
        steamer = pd.read_csv('data/2023-steamer-proj-h.csv', encoding="latin-1")
        zips = pd.read_csv('data/2023-zips-proj-h.csv', encoding="latin-1")
        val_h = pd.read_csv('data/2023-fangraphs-auction-calculator-h.csv')
        val_h.rename(columns={'PlayerId':'playerid', 'POS':'Pos'},inplace=True)
        cbs = pd.read_csv('data/2023-cbs-values.csv', encoding="latin-1")
        
        h = pd.concat([atc,thebatx,steamer,zips]).sort_values('PlayerId')
        h.rename(columns={'PlayerId':'playerid'},inplace=True)
        h = h.merge(val_h[['playerid', 'Pos', 'Dollars']]).merge(cbs[['playerid', 'CBSNAME', 'CBS']], on='playerid', how='left')
        h.drop(columns=['wOBA', 'CS', 'Fld', 'BsR', 'ADP'],inplace=True)
        h['Primary_Pos'] = h.apply(lambda x: find_primary_pos(x['Pos']), axis=1)
        proj = pd.pivot_table(h, index='playerid', values=['G', 'PA', 'AB', 'H', 'HR', 'R', 'RBI', 'SB'], aggfunc='mean').merge(h[['playerid', 'Name', 'CBSNAME', 'Team', 'Pos', 'Primary_Pos', 'Dollars', 'CBS']], on='playerid', how='inner').drop_duplicates()
        proj['sorter'] = proj['HR']+proj['R']+proj['RBI']+proj['H']+proj['SB']
        proj['BA'] = proj['H']/proj['AB']
        proj = proj.drop_duplicates(subset='playerid')

        atc = pd.read_csv('data/2023-atc-proj-p.csv', encoding="latin-1")
        thebat = pd.read_csv('data/2023-thebat-proj-p.csv', encoding="latin-1")
        #dc = pd.read_csv('data/2023-dc-proj-p.csv', encoding="latin-1")
        steamer = pd.read_csv('data/2023-steamer-proj-p.csv', encoding="latin-1")
        zips = pd.read_csv('data/2023-zips-proj-p.csv', encoding="latin-1")
        val_p = pd.read_csv('data/2023-fangraphs-auction-calculator-p.csv')
        val_p.rename(columns={'PlayerId':'playerid', 'POS':'Pos'},inplace=True)
        p = pd.concat([atc,steamer,thebat, zips]).sort_values('PlayerId')
        p.rename(columns={'PlayerId':'playerid'},inplace=True)
        p = p.merge(val_p[['playerid', 'Pos', 'Dollars']]).merge(cbs[['playerid', 'CBSNAME', 'CBS']], on='playerid', how='left')
        p.rename(columns={'H':'HA'},inplace=True)
        p['Sv+Hld'] = p['SV']+p['HLD']
        p['Primary_Pos'] = p['Pos'].apply(lambda x: ', '.join(x.split('/')))

        pproj = pd.pivot_table(p, index='playerid', values=['GS', 'G', 'IP', 'ER', 'HA', 'SO', 'BB', 'W', 'SV', 'HLD', 'Sv+Hld'], aggfunc='mean').merge(p[['playerid', 'Name', 'CBSNAME', 'Team', 'Pos', 'Dollars', 'CBS']], on='playerid', how='inner').drop_duplicates()
        pproj['sorter'] = pproj['SO']+(pproj['Sv+Hld']*4)+pproj['W']
        pproj['Primary_Pos'] = pproj.apply(lambda x: find_primary_pos(x['Pos']), axis=1)
        #proj = proj.append(pproj)
        pproj['IP'].fillna(0, inplace=True)
        for i in ['PA', 'AB', 'G', 'H', 'HR', 'R', 'RBI', 'SB']:
            proj[i].fillna(0,inplace=True)
            proj[i] = proj[i].apply(lambda x: int(x))
        for i in ['GS', 'G', 'HA', 'SO', 'ER', 'BB', 'W', 'SV', 'HLD', 'Sv+Hld']:
            pproj[i].fillna(0,inplace=True)
            pproj[i] = pproj[i].apply(lambda x: int(x))
        pproj['ERA'] = pproj['ER']/pproj['IP']*9
        pproj['WHIP'] = (pproj['HA']+pproj['BB'])/pproj['IP']
        pproj = pproj.drop_duplicates(subset='playerid')

        return proj.sort_values('Dollars', ascending=False), pproj.sort_values('Dollars', ascending=False)
    else:
        #h = batting_stats(datetime.now().year, qual=1)
        stat_list = ['IDfg', 'Name', 'CBSNAME', 'Team', 'G', 'AB', 'PA', 'H', 'HR', 'R', 'RBI', 'BB', 'IBB', 'HBP', 'SF', 'SH', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS']
        all_pos = pd.DataFrame()
        for pos in ['1B', '2B', 'SS', '3B', 'OF', 'C', 'DH']:
            temp = batting_stats(2022, qual=1, position=pos)[stat_list]
            temp['Pos'] = pos
            all_pos = all_pos.append(temp)
        h = all_pos.groupby('IDfg').agg({'Name':'first', 'Team':'first', 'G':'first', 'AB':'first', 'PA':'first', 'H':'first', 'HR':'first', 'R':'first', 'RBI':'first', 
                                                     'BB':'first', 'IBB':'first', 'HBP':'first', 'SF':'first', 'SH':'first', 'SB':'first', 'CS':'first', 'AVG':'first', 'OBP':'first',
                                                     'SLG':'first', 'OPS':'first', 'Pos':'/'.join})
        h['Primary_Pos'] = h.apply(lambda x: find_primary_pos(x['Pos']), axis=1)
        h.rename(columns={'IDfg':'playerid'}, inplace=True)
        p = pitching_stats(datetime.now().year, qual=0)[['IDfg', 'Name', 'CBSNAME', 'Team', 'IP', 'W', 'G', 'GS', 'SV', 'H', 'R', 'ER', 'BB', 'IBB', 'HBP', 'SO', 'HLD', 'Start-IP', 'Relief-IP']]
        p.rename(columns={'H':'HA', 'IDfg':'playerid'}, inplace=True)
        p['Sv+Hld'] = p['SV']+p['HLD']
        p['sorter'] = p['SO']+(p['Sv+Hld']*4)+p['W']
        p.loc[(p['Start-IP'].notna()) & (p['Relief-IP'].isna()), 'Pos'] = 'SP'
        p.loc[(p['Start-IP'].isna()) & (p['Relief-IP'].notna()), 'Pos'] = 'RP'
        p['Pos'].fillna('SP/RP', inplace=True)
        p['Primary_Pos'] = p.apply(lambda x: find_primary_pos(x['Pos']), axis=1)
        return h, p

def calc_z(x, stat):
    z = (x - drafted[stat].mean()) / drafted[stat].std()
    return z

def find_primary_pos(p):
    pos_list = p.split('/')
    pos_hierarchy = ['C', '2B', '3B', 'OF', 'SS', '1B', 'DH', 'SP', 'RP', 'P']
    for i in pos_hierarchy:
        if i in pos_list:
            return i

def owners(conv):
    df = pd.read_sql('players', engine)
    owners_df = df.groupby('Owner').agg({'Name':'count', 'Paid':'sum', 'z':'sum', 'H':'sum', 'AB':'sum', 'HR':'sum', 'R':'sum', 'RBI':'sum', 'SB':'sum', 'W':'sum', 'Sv+Hld':'sum', 'SO':'sum'}).reset_index()
    owners_df.rename(columns={'Name':'Drafted'},inplace=True)
    owners_df['$/unit'] = owners_df['Paid']/owners_df['z']
    owners_df['$ Left'] = tm_dollars - owners_df['Paid']
    owners_df['$ Left / Plyr'] = owners_df['$ Left'] / (tm_players -owners_df['Drafted'])
    owners_df['Cash Sitch'] = owners_df['$ Left / Plyr'] / (((tot_dollars - owners_df.Paid.sum()) + owners_df['Paid']) / ((tot_players - owners_df.Drafted.sum()) + owners_df['Drafted']))
    owners_df['Value'] = (owners_df['z']*conv)-owners_df['Paid']
    owners_df['BA'] = owners_df['H']/owners_df['AB']
    owners_df['Pts'] = 0
    for i in ['BA', 'HR', 'R', 'RBI', 'SB', 'W', 'Sv+Hld', 'SO']:
        owners_df['Pts'] += owners_df[i].rank()
    owners_df['Rank'] = owners_df['Pts'].rank()
    return df.sort_values('z', ascending=False), owners_df

def check_roster_pos(roster, name, team_name, pos, eligible):
    eligible_at = eligible.split('/')
    eligibility = []
    for p in eligible.split('/'):
        if p=='C':
            eligibility.extend(['C'])
        if p=='1B':
            eligibility.extend(['1B', 'CI'])
        if p=='2B':
            eligibility.extend(['2B', 'MI'])
        if p=='3B':
            eligibility.extend(['3B', 'CI'])
        if p=='SS':
            eligibility.extend(['SS', 'MI'])
        if p=='OF':
            eligibility.extend(['OF1', 'OF2', 'OF3', 'OF4', 'OF5'])
        if p in ['SP', 'RP']:
            eligibility.extend(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'])
        
    eligibility = list(dict.fromkeys(eligibility))
    if 'SP' in eligible_at or 'RP' in eligible_at: 
        pos_list = eligibility
    else:
        pos_list = eligibility+['DH1', 'DH2']
    for p in pos_list:
        if roster.loc[p, team_name]==0:
            roster.loc[p, team_name] = name
            return p
    
    return pos_list

def next_closest_in_tier(df, pos, playerid):
    try:
        i = df[(df['Primary_Pos']==pos) & (df['playerid']==playerid) & (df['Owner'].isna())].index[0]
        val = df[(df['Primary_Pos']==pos) & (df['Owner'].isna()) & (df['playerid']==playerid)]['Value'].iloc[0]
        return df[df['playerid']==playerid]['Value'].iloc[0] - df[(df['Primary_Pos']==pos) & (df['Owner'].isna()) & (df['Value']<=val)].iloc[1]['Value']
    except:
        return 0



def get_qual_avgs(year):
    final_h = pd.read_csv('data/'+str(year)+'-final-stats-h.csv').sort_values('PA', ascending=False)
    final_h = final_h[final_h['PA']>440]
    lgBA = final_h['H'].sum()/final_h['AB'].sum()
    final_h['zlgBA'] = final_h.apply(lambda x: x['H']-(x['AB']*(lgBA)), axis=1)
    quals_h = final_h[['H', 'AB', 'PA', 'G', 'zlgBA', 'R', 'RBI', 'HR', 'SB']].describe().to_dict()
    
    final_p = pd.read_csv('data/'+str(year)+'-final-stats-p.csv').sort_values('IP', ascending=False)
    final_p['playerid'] = final_p['playerid'].astype(str)
    final_p['Sv+Hld'] = final_p['SV']+final_p['HLD']
    final_p = final_p.merge(p[['playerid', 'Primary_Pos']], on='playerid', how='inner')
    final_p = final_p[(final_p['Primary_Pos']=='RP') & (final_p['IP'].between(48,90) & (final_p['Sv+Hld']>5)) | (final_p['Primary_Pos']=='SP') & (final_p['IP']>140)]
    lgERA = final_p['ER'].sum()/final_p['IP'].sum()*9
    lgWHIP = (final_p['BB'].sum()+final_p['H'].sum())/final_p['IP'].sum()
    final_p['zlgERA'] = final_p.apply(lambda x: ((x['ER']*9) - (x['IP']*lgERA))*-1, axis=1)
    final_p['zlgWHIP'] = final_p.apply(lambda x: ((x['H']+x['BB'])-(x['IP']*lgWHIP))*-1, axis=1)
    quals_p = final_p[['BB', 'H', 'ER', 'IP', 'SO', 'W', 'Sv+Hld', 'zlgERA', 'zlgWHIP']].describe().to_dict()
    return quals_h, quals_p

qual_avgs = {'G':[145.0, 10.8], 'PA':[600.4, 59.6], 'AB':[533.4, 53.6], 'H':[143.7, 22.5], 'HR':[24.4, 10.4], 'zlgBA':[0.20, 13.9],
            'R':[82.5, 17.6], 'RBI':[78.9, 19.9], 'SB':[8.6, 8.8], 'AVG':[.269, .026], 'W':[6.1, 4.3], 'GS':[7.6, 13.5], 'Sv+Hld':[12.9, 11.6],
            'ER':[37.3, 23.7], 'IP':[91.2, 53.2], 'SO':[95.7, 57.4], 'BB':[30.2, 15.7], 'HA':[79.7, 50.9], 'zlgERA':[.1235, 89.55], 'zlgWHIP':[-.1512, 17.63368]}


def big_board(row, stat):
    # qual_avgs comes from 2019 and 2021 qualified players' averages and stdevs for each stat
    
    if stat == 'BA':
        ba_pts = row['H']-(row['AB']*(qual_h['H']['mean']/qual_h['AB']['mean']))
        zBA = (ba_pts-qual_h['zlgBA']['mean'])/qual_h['zlgBA']['std']
        #return ((row['AB'] * (((row['H']/row['AB'])-qual_avgs['AVG'][0])/qual_avgs['AVG'][1])) - qual_avgs['zlgBA'][0])/qual_avgs['zlgBA'][1]
        return zBA
    elif stat=='ERA':
        pts = ((row['ER']*9) - ((row['IP']*qual_h['ER']['mean']*9)/qual_h['IP']['mean'])) * -1
        zERA = (pts-qual_h['zlgERA']['mean'])/qual_h['zlgERA']['std']
        return zERA
    elif stat=='WHIP':
        pts = ((row['HA']+row['BB'])-(row['IP']*((qual_h['HA']['mean']+qual_h['BB']['mean'])/qual_h['IP']['mean']))) * -1
        zWHIP = (pts-qual_h['zlgWHIP']['mean'])/qual_h['zlgWHIP']['std']
        return zWHIP
    else:
        return (row[stat] - qual_h[stat]['mean']) / qual_h[stat]['std']

In [7]:
ids = load_id_map()

In [8]:
# Set Previous Year
prevYear = datetime.now().year-1

In [9]:
h, p = load_data('proj') # use 'proj' arg for projection info
h.CBS.fillna(0,inplace=True)
p.CBS.fillna(0,inplace=True)

h = h.merge(ids[['IDFANGRAPHS', 'TEAM', 'CBSNAME']], left_on=['playerid', 'Team'], right_on=['IDFANGRAPHS', 'TEAM'], how='left').drop(columns=['IDFANGRAPHS', 'TEAM'])
h.loc[h['CBSNAME_y'].notna(), 'Name'] = h.loc[h['CBSNAME_y'].notna()]['CBSNAME_y']
h['Name'].fillna(h['CBSNAME_x'],inplace=True)
h['Name'].fillna(h['CBSNAME_y'],inplace=True)
h.drop(columns=['CBSNAME_x', 'CBSNAME_y'],inplace=True)

p = p.merge(ids[['IDFANGRAPHS', 'TEAM', 'CBSNAME']], left_on=['playerid', 'Team'], right_on=['IDFANGRAPHS', 'TEAM'], how='left').drop(columns=['IDFANGRAPHS', 'TEAM'])
p.loc[p['CBSNAME_y'].notna(), 'Name'] = p.loc[p['CBSNAME_y'].notna()]['CBSNAME_y']
p['Name'].fillna(p['CBSNAME_x'],inplace=True)
p['Name'].fillna(p['CBSNAME_y'],inplace=True)
p.drop(columns=['CBSNAME_x', 'CBSNAME_y'],inplace=True)

In [10]:
qual_h, qual_p = get_qual_avgs(prevYear)
qual_p['HA'] = qual_p.pop('H')
qual_h.update(qual_p)

In [11]:
h['zR'] = h.apply(lambda row: big_board(row, 'R'), axis=1)
h['zHR'] = h.apply(lambda row: big_board(row, 'HR'), axis=1)
h['zRBI'] = h.apply(lambda row: big_board(row, 'RBI'), axis=1)
h['zSB'] = h.apply(lambda row: big_board(row, 'SB'), axis=1)
h['zBA'] = h.apply(lambda row: big_board(row, 'BA'), axis=1)
h['BIGAA'] = h['zR']+h['zRBI']+h['zHR']+h['zSB']+h['zBA']

p['zSO'] = p.apply(lambda row: big_board(row, 'SO'), axis=1)
p['zW'] = p.apply(lambda row: big_board(row, 'W'), axis=1)
p['zSv+Hld'] = p.apply(lambda row: big_board(row, 'Sv+Hld'), axis=1)
p['zERA'] = p.apply(lambda row: big_board(row, 'ERA'), axis=1)
p['zWHIP'] = p.apply(lambda row: big_board(row, 'WHIP'), axis=1)
p['BIGAA'] = p['zSv+Hld']+p['zSO']+p['zW']+p['zERA']+p['zWHIP']

print('Positional adjustment applied by primary position')
c_adjust = abs(h[h['Primary_Pos']=='C'].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['C']]['BIGAA'])
h.loc[h['Primary_Pos']=='C', 'Pos_adj'] = c_adjust
ci_adjust = abs(h[h['Primary_Pos'].isin(['1B', '3B'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['1B']+drafted_by_pos['3B']]['BIGAA'])
h.loc[h['Primary_Pos'].isin(['1B', '3B']), 'Pos_adj'] = ci_adjust
mi_adjust = abs(h[h['Primary_Pos'].isin(['2B', 'SS'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['SS']+drafted_by_pos['2B']]['BIGAA'])
h.loc[h['Primary_Pos'].isin(['2B', 'SS']), 'Pos_adj'] = mi_adjust
of_adjust = abs(h[h['Primary_Pos'].isin(['OF', 'DH'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['OF']]['BIGAA'])
h.loc[h['Primary_Pos'].isin(['OF', 'DH']), 'Pos_adj'] = of_adjust

print('C',h[h['Primary_Pos']=='C'].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['C']]['BIGAA'])
print('MI',h[h['Primary_Pos'].isin(['2B', 'SS'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['1B']+drafted_by_pos['3B']]['BIGAA'])
print('CI',h[h['Primary_Pos'].isin(['1B', '3B'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['SS']+drafted_by_pos['2B']]['BIGAA'])
print('OF',h[h['Primary_Pos'].isin(['OF', 'DH'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['OF']]['BIGAA'])

sp_adjust = abs(p[p['Primary_Pos']=='SP'].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['SP']]['BIGAA'])
p.loc[p['Primary_Pos']=='SP', 'Pos_adj'] = sp_adjust
rp_adjust = abs(p[p['Primary_Pos']=='RP'].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['RP']]['BIGAA'])
p.loc[p['Primary_Pos']=='RP', 'Pos_adj'] = rp_adjust
print('SP',p[p['Primary_Pos'].isin(['SP'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['SP']]['BIGAA'])
print('RP',p[p['Primary_Pos'].isin(['RP'])].sort_values('BIGAA',ascending=False).iloc[drafted_by_pos['RP']]['BIGAA'])

# Apply Positional adjustment
h['z'] = h['BIGAA'] + h['Pos_adj']
p['z'] = p['BIGAA'] + p['Pos_adj']

conv = (tm_dollars/tm_players)*(tot_players/(h[h['z']>0]['z'].sum()+p[p['z']>0]['z'].sum()))
print('\nTotal z:',h[h['z']>0]['z'].sum()+p[p['z']>0]['z'].sum())
print('\nH/P split:',h[h['z']>0]['z'].sum()/(h[h['z']>0]['z'].sum()+p[p['z']>0]['z'].sum()))
print('Conversion to $:',conv)

h['Value'] = h['z']*conv
p['Value'] = p['z']*conv

p.loc[p['Name']=='Shohei Ohtani', 'playerid'] = 'p19755'

b = h.append(p)
b['Outs'] = b['IP']*3
b['K/9'] = b['SO']*9/(b['Outs']/3)
b['Timestamp'] = None
b['Paid'] = 0
b['Owner'] = None
b['Used'] = None

Positional adjustment applied by primary position
C -2.715431109736751
MI -1.0443912249793308
CI -2.086855283386387
OF -1.632998635565257
SP -1.8630275082670624
RP -0.83523002274616

Total z: 626.1342270115357

H/P split: 0.6769413674645359
Conversion to $: 4.9829571127126995


In [ ]:
# Insert new projection data into players table
#b.to_sql('players', engine, if_exists='replace')

In [ ]:
b['Value-CBS'] = b['Value'] - b['CBS']

In [ ]:
c = b[b['Name'].isna()].merge(ids[['IDFANGRAPHS', 'CBSNAME']], left_on='playerid', right_on='IDFANGRAPHS', how='inner')
c['Name'].fillna(c['CBSNAME'],inplace=True)
c.rename(columns={'Name':'new name'},inplace=True)
b = b.merge(c[['playerid','new name']], on='playerid', how='left')
b.Name.fillna(b['new name'],inplace=True)
b.drop(columns='new name', inplace=True)
b.Name.isna().sum()

In [ ]:
b.sort_values('Value-CBS', ascending=False)[['playerid', 'Name', 'Value', 'CBS', 'Value-CBS', 'BA', 'HR', 'R', 'RBI', 'SB', 'Team', 'Pos']].head(25)

In [ ]:
b[b['Primary_Pos'].isin(['OF'])].sort_values('z', ascending=False).head(18)[['Name', 'z', 'Value', 'CBS', 'Dollars', 'HR', 'SB', 'R', 'RBI', 'BA']].reset_index()

### Copy Data from Last Year and Create Fresh Table

In [ ]:
# First check available tables
pd.read_sql("SELECT name FROM sqlite_master", engine)

In [ ]:
# Create new table <name>
#pd.read_sql("CREATE TABLE players2023 AS SELECT * FROM players",engine)

In [ ]:
# Drop table if needed
#pd.read_sql('DROP TABLE players2022',engine)

In [ ]:
# Check the old table with new name
pd.read_sql("SELECT * from players2022", engine).shape

### Update data with Keepers

In [ ]:
# Keepers
keepers = {
    'playerid':['26197', '19556', '14374', '9803', '17295', '21711', '20970', '25385', '19361', '15518', '11156', '15454',
               '13510', '30116', '21390', '19959', '17606', '26288'],
    'Owner':['Young Guns', 'Young Guns', 'Young Guns', 'Young Guns', 'Ugly Spuds', 'Ugly Spuds', 'Ugly Spuds', 
             'Ugly Spuds', 'Trouble', 'Trouble', 'Trouble', 'Trouble', 'Charmer', 'Charmer', 'Charmer', 'Charmer', 'Lima Time',
            'Lima Time'],
    'Paid':[1, 30, 0, 1, 9, 1, 2, 0, 34, 1, 1, 0, 38, 4, 1, 0, 0, 2],
}

kdf = pd.DataFrame(keepers)
kdf['Timestamp'] = datetime.now()
kdf['Keeper'] = 1
kdf

In [ ]:
b.Name.fillna('Unk',inplace=True)
b[b['Name'].str.contains('Amed')]

In [ ]:
b[b['Owner'].notna()][['playerid', 'Name', 'Owner', 'Paid']].sort_values('Owner')

In [ ]:
pd.read_sql('SELECT * FROM players WHERE Keeper=1', engine).sort_values('Owner')[['playerid', 'Name', 'Owner', 'Paid', 'Keeper']]

In [ ]:
b = b.drop(columns=['Paid', 'Timestamp', 'Owner']).merge(kdf, on='playerid', how='left')
b['Paid'].fillna(0,inplace=True)
b["Paid"] = pd.to_numeric(b.Paid, downcast='integer')
b['Keeper'].fillna(0,inplace=True)

In [ ]:
# Uploads data to table
#b.set_index('playerid')[['Name', 'Team', 'Pos', 'Primary_Pos', 'G', 'GS', 'PA', 'AB', 'H', 'HR', 'R', 'RBI', 'SB', 'BA', 'IP', 'Outs', 'HA', 'BB', 'ER', 'W', 'SO', 'Sv+Hld', 'ERA', 'WHIP', 'K/9', 'Dollars', 'z', 'Value', 'Paid', 'Owner', 'Used', 'Timestamp', 'zBA', 'zHR', 'zR', 'zRBI', 'zSB', 'zERA', 'zWHIP', 'zW', 'zSO', 'zSv+Hld']]
b.to_sql('players', engine, if_exists='replace')

### Working with Drafted Players

In [ ]:
df, owners_df = owners(conv)
owners_df

In [ ]:
owners_df['Value'] = owners_df['z']*conv

In [ ]:
owner_list = ['Brewbirds', 'Charmer', 'Dirty Birds', 'Harvey', 'Lil Trump', 'Lima Time', 'Midnight', 'Ugly Spuds', 'Roid Ragers', 'Trouble', 'Wu-Tang', 'Young Guns']
roster = pd.DataFrame(index=['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'], data=np.zeros((23,12)), columns=owner_list)

In [ ]:
owner_list = ['Brewbirds', 'Charmer', 'Dirty Birds', 'Harvey', 'Lil Trump', 'Lima Time', 'Midnight', 'Ugly Spuds', 'Roid Ragers', 'Trouble', 'Wu-Tang', 'Young Guns']
roster = pd.DataFrame(index=['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'], data=np.zeros((23,12)), columns=owner_list)
for tm in owners_df.Owner.to_list():
    for i, row in df[df['Owner']==tm][['Name', 'Owner', 'Primary_Pos', 'Pos', 'Timestamp']].sort_values("Timestamp").iterrows():
        if df.loc[i]['Paid'] > 0:
            check_roster_pos(roster, df.loc[i]['Name'], df.loc[i]['Owner'], df.loc[i]['Primary_Pos'], df.loc[i]['Pos'])

In [ ]:
roster

In [ ]:
print('# of Players Yet to be Drafted:',(tot_players) - owners_df.Drafted.sum())
print('$ still available to be spent:', tot_dollars - owners_df.Paid.sum())
print('Initial $ per z rate:',tot_dollars/df[df['z']>=0]['z'].sum())
print('Current $ per z rate:',owners_df.Paid.sum() / owners_df.z.sum())
conv_factor = (tot_dollars - owners_df.Paid.sum()) / (df[df['z']>0]['z'].sum()-owners_df['z'].sum())
print('Conversion Factor:',conv_factor)

In [ ]:
def set_color(x):
    if(x == None):
        return "blue"
    else:
        return "gray"

        
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df['Name'].head(tot_players+25),
        y=df['z'].head(tot_players+25),
        mode='markers',
        marker=dict(color=list(map(set_color,df['Owner'])))
    )
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=b['Primary_Pos'],
        y=b.query('z>=0')['z'],
        text=b['Name'],
        mode='markers',
    )
)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(name='3B', x=b[b['Pos'].str.contains('3B')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))
fig.add_trace(go.Histogram(name='1B', x=b[b['Pos'].str.contains('1B')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))
fig.add_trace(go.Histogram(name='2B', x=b[b['Pos'].str.contains('2B')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))
fig.add_trace(go.Histogram(name='SS', x=b[b['Pos'].str.contains('SS')].sort_values('Value', ascending=False).drop_duplicates('playerid').head(300)['z'], 
                           nbinsx=10, opacity=.5))

fig.update_layout(title='Z score distribution by position', barmode='overlay', bargap=.1, yaxis_range=[0,55])
fig.show()

In [ ]:
b.groupby('Primary_Pos')['Pos_adj'].mean()

In [ ]:
b[b['Pos'].str.contains('SP')].sort_values('z',ascending=False).head(25)[['Name', 'z', 'Value']]

#### Next Closest in Tier

In [ ]:
tier_pos = df[df['Name']=='Freddie Freeman']['Primary_Pos'].iloc[0]
drop_in_tier = round(df[(df['Primary_Pos']==tier_pos) & (df['Owner'].isna())].iloc[1]['Value'] - df[(df['Primary_Pos']==tier_pos) & (df['Owner'].isna())].iloc[0]['Value'],1)
print(drop_in_tier)
df[(df['Primary_Pos']==tier_pos) & (df['Owner'].isna())].iloc[:6][['Name', 'Pos', 'z', 'Value', 'HR', 'SB', 'RBI', 'R', 'BA']]

In [ ]:
i = df[(df['Primary_Pos']=='SS') & (df['playerid']=='18314') & (df['Owner'].isna())].index[0]
v = df[(df['Primary_Pos']=='SS') & (df['Owner'].isna()) & (df['playerid']=='18314')]['Value'].iloc[0]
df[(df['Primary_Pos']=='SS') & (df['Owner'].isna()) & (df['Value']<=v)].iloc[1]['Value']

In [ ]:
df['next_in_tier'] = df.apply(lambda x: next_closest_in_tier(df, x['Primary_Pos'], x['playerid']),axis=1)

In [ ]:
df[(df['Primary_Pos']==tier_pos) & (df['z']>0)]['Value'].hist()

### StatCast work

In [ ]:
sc = statcast('2019-03-01', '2021-11-30')

In [ ]:
sc.columns

In [ ]:
sc[(sc['batter']==645277) & (sc['launch_speed'].notna())]['launch_speed'].sort_values()

In [ ]:
sc.groupby('batter')['launch_speed'].max().sort_values(ascending=False).head(45)

In [ ]:
sc = pd.read_csv('data/statcast-exit_velocity.csv')
sc = sc.sort_values('brl_pa', ascending=False)
sc = sc.merge(df[['playerid', 'Name', 'Primary_Pos']], on='playerid', how='left')
sc.Name.fillna(sc['first_name']+' '+sc['last_name'],inplace=True)
sc = sc[~sc['Primary_Pos'].isin(['SP', 'RP'])]
sc['brl_pa_rank'] = sc.brl_pa.rank(pct=True)

In [ ]:
sc['brl_pa_rank'] = sc.brl_pa.rank(pct=True)

In [ ]:
sc.loc[850]

In [ ]:
pd.pivot_table(columns='year', data=sc, index='Name')

## ------ End of New Work

In [ ]:
from sqlalchemy import text
pid = '19755'
price = 14
owner = 'Wu-Tang'
timestamp = None
t = text("UPDATE players SET Owner='"+owner+"', Paid="+str(price)+" WHERE playerid='"+pid+"'")
#t = text("UPDATE players SET Owner=null, Paid=null, Timestamp=null WHERE playerid='"+pid+"'")
#'Timestamp', 'Paid', 'Owner'
result = conn.execute(t)
result

https://leportella.com/sqlalchemy-tutorial/

In [ ]:
#Creates a table on the DB
meta = MetaData()
hitters = Table('hitting', meta,
                Column('playerid', String, primary_key=True),
                #Column('name', String(50)),
                #Column('team', String(20)),
                #Column('pa', Integer),
                #Column('ab', Integer),
                #Column('ba', Float),
                #Column('h', Integer),
                #Column('hr', Integer),
                #Column('r', Integer),
                #Column('rbi', Integer),
                #Column('sb', Integer),
                Column('Paid', Integer),
                Column('Owner', String(25)),
                Column('Used', Boolean)
)
meta.create_all(engine)

In [ ]:
#conn.execute(hitters.delete().where(hitters.c.Used==None))

In [ ]:
conn.execute(hitters.update().values(Paid=15, Owner='Harvey').where(hitters.c.playerid=='15172'))
conn.execute(hitters.update().values(Paid=1, Owner='Harvey').where(hitters.c.playerid=='16375'))

In [ ]:
single_player = (h['Name']=='Robert Witt')

fig = go.Figure(data=go.Scatterpolar(
  r=h[single_player][['zBA', 'zR', 'zRBI', 'zHR', 'zSB']].values.tolist()[0],
  theta=['zBA', 'zR', 'zRBI', 'zHR', 'zSB'],
  fill='toself',
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()

In [ ]:
fig = px.scatter(
    owners_df, x='$ Left', y='Pts', opacity=0.65,
    trendline='ols', trendline_color_override='darkblue'
)
fig.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.stats.stats import pearsonr
def scale_data(df, cols):
    """
    INPUT: 
        df: original dataframe
        list: subset of columns to scale
    OUTPUT:
        df: scaled data
    """
    scaler = MinMaxScaler()
    scaler.fit(df[cols])
    scaled_df = scaler.transform(df[cols])
    scaled_df = pd.DataFrame(scaled_df, index=df.index)
    scaled_df.columns=[df[cols].columns.tolist()]
    return scaled_df

In [ ]:
cols = ['BA', 'R', 'RBI', 'HR', 'SB']
scaled_df = scale_data(df[df['Owner'].isna()].set_index('playerid'), cols)
scaled_df.loc['11737', cols]

In [ ]:
def add_distance_metrics(h, player_id, col_list):
    scaled_df = scale_data(h[h['Owner'].isna()].set_index('playerid'), col_list)
    df2 = h[h['Owner'].isna()].loc[:,['playerid', 'Name', 'Pos', 'z']+col_list].set_index('playerid')
    for j, row in scaled_df.iterrows():
        #df2.at[j,'corr'] = pearsonr(scaled_df.loc[player_id,col_list],row[col_list])[0]
        df2.at[j,'eucl_dist'] = np.linalg.norm(scaled_df.loc[player_id,col_list] - row[col_list])
        #df2.at[j,'manh_dist']= sum(abs(e - s) for s, e in zip(scaled_df.loc[player_id,col_list], row[col_list]))
    return df2.sort_values('eucl_dist').iloc[1:6]

In [ ]:
add_distance_metrics(df, '9218', cols)

In [ ]:
mask = (df['Pos'].str.contains('C')) & (df['Owner'].isna())
df[mask][['Name', 'z', 'Dollars', 'HR', 'R', 'RBI', 'SB', 'BA']].sort_values('z', ascending=False).head(10)

In [ ]:
# (Tm 1 $/plyr left / Tm 2 $/plyr left) * suggested price = equivalent comfort level
pp = 5
f'If the price is {pp}, Tm 1 can afford to pay up to {round((10.125 / 6.9) * pp,1)} at the same comfort level as Tm 2'

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=b['Dollars'],
    y=b['Value'],
    mode='markers',
    text=b['Name']
)
)
fig.show()

In [ ]:
#Creates a table on the DB
meta = MetaData()
players = Table('players', meta,
                Column('playerid', String, primary_key=True),
                Column('Name', String(50)),
                Column('Team', String(20)),
                Column('Pos', String(30)),
                Column('Primary_Pos', String(10)),
                Column('G', Integer),
                Column('GS', Integer),
                Column('PA', Integer),
                Column('AB', Integer),
                Column('H', Integer),
                Column('HR', Integer),
                Column('R', Integer),
                Column('RBI', Integer),
                Column('SB', Integer),
                Column('BA', Float),
                Column('OBP', Float),
                Column('SLG', Float),
                Column('OPS', Float),
                Column('IP', Float),
                Column('Outs', Integer),
                Column('HA', Integer),
                Column('BB', Integer),
                Column('ER', Integer),
                Column('SV', Integer),
                Column('HLD', Integer),
                Column('Sv+Hld', Integer),
                Column('W', Integer),
                Column('SO', Integer),
                Column('ERA', Float),
                Column('WHIP', Float),
                Column('K/9', Float),
                Column('Dollars', Float),
                Column('CBS', Integer),
                Column('z', Float),
                Column('Value', Float),
                Column('Paid', Integer),
                Column('Owner', String(25)),
                Column('Used', Boolean),
                Column('Timestamp', DateTime)
)
meta.create_all(engine)

In [ ]:
conn.execute('ALTER TABLE hitting ADD COLUMN WHIP Float')

In [ ]:
#conn.execute('DROP Table players')

In [ ]:
#conn.execute('ALTER TABLE hitting DROP COLUMN Paid2')

In [ ]:
#conn.execute('ALTER TABLE hitting ADD COLUMN Timestamp DATETIME')

In [ ]:
conn.close()

### Fixing 2022 Draft in sqlite db

In [ ]:
url = 'https://xdl.baseball.cbssports.com/draft/results/2022:Pre-season:XDL%20Draft'

In [ ]:
df = pd.read_sql('players', engine)
df.Owner.unique()

In [ ]:
pd.read_sql('players2022', engine).Owner.value_counts()

In [ ]:
tm = 'Lima Time'
df = pd.read_sql('players', engine)
print(df[df['Owner']==tm][['playerid', 'Name', 'Paid']].shape[0])
df[df['Owner']==tm][['playerid', 'Name', 'Paid', 'Value']].sort_values('Paid',ascending=False)

In [ ]:
df.groupby('Owner')['Name'].count()

In [ ]:
url = 'https://xdl.baseball.cbssports.com/features/projected-salaries'
pd.read_html(url, match='SORTABLE')

In [ ]:
def fix_downloaded_cbs_values(year, save=False):
    # First get spreadsheet from CBS auction values page
    # Then parse name out of first column
    # Save it to data/ folder as yyyy-cbs-values.csv
    ids = load_id_map()
    cbs = pd.read_csv('data/'+str(year)+'-cbs-values.csv')
    cbs = cbs.merge(ids[['CBSNAME', 'TEAM', 'IDFANGRAPHS']], left_on=['Name', 'Team'], right_on=['CBSNAME','TEAM'], how='left')
    cbs.rename(columns={'IDFANGRAPHS':'playerid'},inplace=True)
    if save:
        cbs[['playerid', 'Name', 'Pos', 'Team', 'CBS']].to_csv('data/'+str(year)+'-cbs-values.csv',index=False)
    return cbs

In [ ]:
cbs = fix_downloaded_cbs_values(2023, save=True)
#cbs = pd.read_csv('data/'+str(2023)+'-cbs-values.csv')

In [16]:
from pybaseball import statcast_batter_exitvelo_barrels, statcast_batter_expected_stats, statcast_batter_percentile_ranks

# get data for all qualified batters in 2019
d1 = statcast_batter_exitvelo_barrels(2022).set_index('player_id')
d2 = statcast_batter_expected_stats(2022).set_index('player_id')
d3 = statcast_batter_percentile_ranks(2022).set_index('player_id')

In [25]:
d2.columns, d3.columns

(Index(['last_name', 'first_name', 'year', 'pa', 'bip', 'ba', 'est_ba',
        'est_ba_minus_ba_diff', 'slg', 'est_slg', 'est_slg_minus_slg_diff',
        'woba', 'est_woba', 'est_woba_minus_woba_diff'],
       dtype='object'),
 Index(['player_name', 'year', 'xwoba', 'xba', 'xslg', 'xiso', 'xobp', 'brl',
        'brl_percent', 'exit_velocity', 'hard_hit_percent', 'k_percent',
        'bb_percent', 'whiff_percent', 'sprint_speed', 'oaa'],
       dtype='object'))

In [28]:
data = d1.merge(d2.drop(columns=['last_name', 'year']), left_index=True, right_index=True, how='left')\
    .merge(d3, left_index=True, right_index=True, how='left')

In [29]:
data

,last_name,first_name,attempts,avg_hit_angle,anglesweetspotpercent,max_hit_speed,avg_hit_speed,fbld,gb,max_distance,avg_distance,avg_hr_distance,ev95plus,ev95percent,barrels,brl_percent_x,brl_pa,first_name,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,player_name,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent_y,exit_velocity,hard_hit_percent,k_percent,bb_percent,whiff_percent,sprint_speed,oaa
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
543760,Semien,Marcus,547,19.9,32.4,110.1,87.3,91.3,84.2,430,186,394.0,191,34.9,37,6.8,5.1,Marcus,724,547,0.248,0.243,0.005,0.429,0.394,0.035,0.317,0.306,0.011,Marcus Semien,2022,38.0,42.0,47.0,51.0,30.0,74.0,41.0,18.0,22.0,79.0,39.0,76.0,89,95.0
642708,Rosario,Amed,530,5.0,31.1,110.8,88.4,92.1,86.9,450,138,407.0,203,38.3,24,4.5,3.6,Amed,670,530,0.283,0.276,0.007,0.403,0.396,0.007,0.311,0.310,0.001,Amed Rosario,2022,43.0,89.0,48.0,24.0,39.0,41.0,18.0,39.0,41.0,79.0,2.0,74.0,96,2.0
608070,Ramírez,José,528,20.7,33.7,114.2,87.7,91.2,86.9,422,179,392.0,195,36.9,35,6.6,5.3,José,665,528,0.280,0.255,0.025,0.514,0.408,0.106,0.363,0.320,0.043,José Ramírez,2022,54.0,62.0,55.0,53.0,75.0,72.0,39.0,25.0,32.0,95.0,75.0,94.0,81,85.0
607208,Turner,Trea,527,10.2,35.3,112.5,88.9,92.1,86.3,439,163,402.0,219,41.6,40,7.6,5.7,Trea,707,527,0.298,0.276,0.022,0.466,0.432,0.034,0.350,0.335,0.015,Trea Turner,2022,71.0,89.0,72.0,57.0,61.0,78.0,48.0,48.0,60.0,68.0,25.0,39.0,99,48.0
665489,Guerrero Jr.,Vladimir,526,4.3,27.9,118.4,92.8,98.2,90.3,467,144,407.0,265,50.4,59,11.2,8.4,Vladimir,700,526,0.274,0.276,-0.002,0.480,0.462,0.018,0.351,0.348,0.003,Vladimir Guerrero Jr.,2022,83.0,89.0,84.0,75.0,81.0,96.0,79.0,96.0,94.0,81.0,49.0,51.0,34,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623205,Velazquez,Andrew,212,8.2,31.1,109.1,88.0,93.3,83.1,402,146,380.0,72,34.0,14,6.6,4.0,Andrew,349,212,0.196,0.202,-0.006,0.304,0.312,-0.008,0.239,0.246,-0.007,Andrew Velazquez,2022,1.0,2.0,3.0,18.0,1.0,16.0,39.0,33.0,18.0,2.0,7.0,7.0,93,81.0
665833,Cruz,Oneil,206,8.3,30.6,122.4,91.9,97.2,88.8,437,152,411.0,95,46.1,32,15.5,8.9,Oneil,360,206,0.233,0.223,0.010,0.450,0.410,0.040,0.320,0.304,0.016,Oneil Cruz,2022,36.0,17.0,58.0,75.0,14.0,63.0,96.0,91.0,81.0,1.0,46.0,4.0,98,3.0
666915,Dalbec,Bobby,203,15.2,32.0,113.9,90.3,93.3,87.4,436,177,396.0,93,45.8,24,11.8,6.8,Bobby,353,203,0.215,0.216,-0.001,0.369,0.379,-0.010,0.287,0.296,-0.009,Bobby Dalbec,2022,24.0,8.0,38.0,61.0,15.0,41.0,84.0,75.0,79.0,2.0,49.0,5.0,76,40.0


In [35]:
d1.sort_values('brl_percent', ascending=False)

,last_name,first_name,attempts,avg_hit_angle,anglesweetspotpercent,max_hit_speed,avg_hit_speed,fbld,gb,max_distance,avg_distance,avg_hr_distance,ev95plus,ev95percent,barrels,brl_percent,brl_pa
player_id,,,,,,,,,,,,,,,,,
592450,Judge,Aaron,400,15.0,39.0,118.4,95.9,100.3,89.2,465,206,412.0,247,61.8,106,26.5,15.7
670541,Alvarez,Yordan,371,12.3,40.7,117.4,95.2,98.1,91.7,469,196,405.0,222,59.8,78,21.0,14.1
656941,Schwarber,Kyle,379,19.2,35.4,114.8,93.3,99.3,87.0,468,200,412.0,206,54.4,76,20.1,11.4
545361,Trout,Mike,300,24.6,37.7,114.4,91.6,94.8,86.7,490,219,409.0,153,51.0,59,19.7,12.0
519317,Stanton,Giancarlo,264,10.8,26.9,119.8,95.0,98.4,94.4,457,163,402.0,138,52.3,51,19.3,11.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624428,Frazier,Adam,478,13.6,37.4,105.1,85.1,88.9,82.5,403,159,344.0,117,24.5,7,1.5,1.2
680757,Kwan,Steven,509,11.8,34.6,107.1,85.1,87.4,83.9,400,155,381.0,106,20.8,7,1.4,1.1
643396,Kiner-Falefa,Isiah,419,4.3,29.4,109.8,86.2,90.1,85.1,404,125,378.0,126,30.1,5,1.2,0.9


In [40]:
d4.head(8)

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
6,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,28,0,62,133,131,111,19,175,6,5,0,14,16,3,0.311,149,174,77,8,2906,1176,1730,11,0,0,0.159,0.251,0.63,0.425,0.686,1.111,0.375,0.340,0.0086,0.193,0.373,0.435,0.046,0.356,0.074,0.000,0.458,82.1,162,84.0,4.7,20.8,-4.1,108.8,11.4,$91.4,4.0,207,7.74,-10.71,18.44,82.07,8.81,1.08,2.54,4,8.60,-1.43,0.437,94.1,0.258,84.6,0.072,88.1,0.077,80.6,0.124,85.1,0.032,88.2,NaN,NaN,0.023,NaN,43.0,24.2,1.3,5.2,5.6,-1.1,NaN,3.38,3.24,0.63,2.31,1.54,-1.20,NaN,0.268,0.676,0.425,0.517,0.852,0.722,0.385,0.579,0.118,2.1,0.275,NaN,0.068,0.023,NaN,0.160,0.257,0.070,0.012,0.001,0.134,NaN,NaN,NaN,94.4,NaN,87.8,88.3,NaN,93.6,85.0,79.7,80.9,51.6,85.2,NaN,NaN,-2.3,NaN,1.3,-6.8,NaN,-6.3,2.6,2.4,3.4,-4.3,0.2,NaN,NaN,9.8,NaN,4.4,2.3,NaN,5.0,1.4,-4.8,-6.8,7.2,4.4,NaN,NaN,25.4,NaN,-1.4,-0.5,NaN,16.0,25.5,4.0,1.8,0.9,4.7,NaN,NaN,3.18,NaN,-0.69,-0.68,NaN,3.44,3.41,2.00,4.95,45.60,1.20,NaN,NaN,0.229,0.649,0.427,0.450,0.829,0.722,0.471,24.5,0.5,1.2,0.4,30 - 30,86.1,1.4,0.5,0.475,0.318,0.208,0.075,0.445,0.480,0.500,0.133,NaN,0.059,0.277,0.065,0.024,NaN,NaN,0.159,0.280,0.002,85.1,NaN,79.8,94.3,87.9,87.8,NaN,NaN,93.8,84.5,85.3,0.2,NaN,2.2,-1.9,1.5,-6.4,NaN,NaN,-6.1,3.3,1.1,3.1,NaN,-7.7,8.8,3.4,1.3,NaN,NaN,3.9,0.2,0.3,6.5,NaN,5.9,24.6,0.1,-1.0,NaN,NaN,15.4,24.3,-0.5,1.68,NaN,3.43,3.05,0.07,-1.42,NaN,NaN,3.33,2.98,-8.22,0.224,0.654,0.427,0.452,0.826,0.722,0.471,24.5,NaN,128.0,200,113,137,175.0,251.0,117.0,0.97,87.0,116.0,324.0,118.0,91.0,84.0,46.0,83.0,159.0,95.8,14.9,106,0.262,118.4,246,0.609,404,0.169,0.287,0.305,0.706,0.463,11.2
23,11493,2022,Manny Machado,SDP,29,1

In [44]:
d2.sort_values('est_woba')

,last_name,first_name,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
player_id,,,,,,,,,,,,,,
645302,Robles,Victor,2022,407,277,0.224,0.203,0.021,0.311,0.280,0.031,0.260,0.244,0.016
623205,Velazquez,Andrew,2022,349,212,0.196,0.202,-0.006,0.304,0.312,-0.008,0.239,0.246,-0.007
455117,Maldonado,Martín,2022,379,234,0.186,0.181,0.005,0.352,0.325,0.027,0.264,0.253,0.011
656308,Chavis,Michael,2022,425,279,0.229,0.217,0.012,0.389,0.336,0.053,0.282,0.261,0.021
545358,Stassi,Max,2022,375,223,0.180,0.180,0.000,0.303,0.325,-0.022,0.258,0.266,-0.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545361,Trout,Mike,2022,491,300,0.283,0.265,0.018,0.630,0.583,0.047,0.418,0.395,0.023
665742,Soto,Juan,2022,658,428,0.242,0.266,-0.024,0.452,0.501,-0.049,0.376,0.401,-0.025
518692,Freeman,Freddie,2022,696,517,0.325,0.313,0.012,0.511,0.538,-0.027,0.393,0.403,-0.010


In [42]:
d3.sort_values('xwoba', ascending=False).dropna().head(15)

,player_name,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,exit_velocity,hard_hit_percent,k_percent,bb_percent,whiff_percent,sprint_speed,oaa
player_id,,,,,,,,,,,,,,,,
592450,Aaron Judge,2022,100.0,99.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,25.0,99.0,18.0,50,85.0
518692,Freddie Freeman,2022,99.0,100.0,98.0,91.0,99.0,92.0,68.0,88.0,88.0,90.0,90.0,80.0,39,67.0
665742,Juan Soto,2022,99.0,80.0,95.0,94.0,99.0,94.0,86.0,85.0,86.0,90.0,100.0,83.0,33,1.0
545361,Mike Trout,2022,98.0,78.0,99.0,99.0,90.0,96.0,99.0,90.0,96.0,12.0,81.0,17.0,95,83.0
656941,Kyle Schwarber,2022,97.0,31.0,98.0,99.0,80.0,99.0,99.0,99.0,99.0,8.0,94.0,22.0,29,1.0
663586,Austin Riley,2022,97.0,85.0,98.0,98.0,86.0,98.0,96.0,96.0,95.0,30.0,49.0,30.0,61,5.0
608369,Corey Seager,2022,96.0,96.0,97.0,91.0,90.0,93.0,73.0,86.0,78.0,85.0,57.0,38.0,24,86.0
547989,José Abreu,2022,96.0,98.0,92.0,76.0,98.0,88.0,63.0,93.0,97.0,82.0,61.0,58.0,20,64.0
592626,Joc Pederson,2022,95.0,80.0,96.0,95.0,85.0,82.0,95.0,98.0,98.0,39.0,67.0,51.0,20,2.0


In [38]:
d4 = batting_stats(2022, qual=1)

In [45]:
list(d4.columns)

['IDfg',
 'Season',
 'Name',
 'Team',
 'Age',
 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'GDP',
 'SB',
 'CS',
 'AVG',
 'GB',
 'FB',
 'LD',
 'IFFB',
 'Pitches',
 'Balls',
 'Strikes',
 'IFH',
 'BU',
 'BUH',
 'BB%',
 'K%',
 'BB/K',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'wOBA',
 'wRAA',
 'wRC',
 'Bat',
 'Fld',
 'Rep',
 'Pos',
 'RAR',
 'WAR',
 'Dol',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'phLI',
 'PH',
 'WPA/LI',
 'Clutch',
 'FB% (Pitch)',
 'FBv',
 'SL%',
 'SLv',
 'CT%',
 'CTv',
 'CB%',
 'CBv',
 'CH%',
 'CHv',
 'SF%',
 'SFv',
 'KN%',
 'KNv',
 'XX%',
 'PO%',
 'wFB',
 'wSL',
 'wCT',
 'wCB',
 'wCH',
 'wSF',
 'wKN',
 'wFB/C',
 'wSL/C',
 'wCT/C',
 'wCB/C',
 'wCH/C',
 'wSF/C',
 'wKN/C',
 'O-Swing%',
 'Z-Swing%',
 'Swing%',
 'O-Contact%',
 'Z-Contact%',
 'Contact%',
 'Zone%',
 'F-Strike%',
 'SwStr%',
 'BsR',
 'FA% (sc)',
 